In [1]:
import pandas as pd
import numpy as np
import csv
import networkx
from networkx.algorithms.components.connected import connected_components

def to_graph(l):
    G = networkx.Graph()
    for part in l:
        G.add_nodes_from(part)
        G.add_edges_from(to_edges(part))
    return G

def to_edges(l):
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current   

def get_index(mydict, mylist):
    for key,values in mydict.items():
        if key != '':
            mylist.append(list(values.index))
            
data = pd.read_json('contacts.json')

email = dict(list(data.groupby(['Email'])))
phone = dict(list(data.groupby(['Phone'])))
orderid = dict(list(data.groupby(['OrderId'])))

storage = []

get_index(email, storage)
get_index(phone, storage)
get_index(orderid, storage)

k = np.zeros((500000,20))

for item in storage:
    for element1 in item:
        i = 0
        for element2 in item:
            while(k[element1,i] != 0):
                i +=1
            k[element1,i] = element2+1

new = list(k)

for i in range(len(new)):
    new[i] = list(set(new[i]))
    new[i].remove(0)
    new[i].sort()
    for j in range(len(new[i])):
        new[i][j] = int(new[i][j] - 1)
        new[i][j] = str(new[i][j])

G = to_graph(new)
contact = []
contact = data['Contacts'].to_numpy()
newlist = []

for item in connected_components(G):
    item = list(item)
    item.sort(key = lambda x:int(x))
    newlist.append(item)
    contact[list(map(int, item))] = sum(data.loc[int(x),'Contacts'] for x in item)

finalarr = np.empty((500000,2),dtype=object)

for item in newlist:
    mystr = '-'.join(element for element in item)
    mystr = mystr +', ' + str(contact[int(item[0])])
    for element in item:
        element = int(element)
        finalarr[element,0] = str(element)
        finalarr[element,1] = mystr

final = pd.DataFrame(finalarr,columns = ['ticket_id','ticket_trace/contact'])
final.to_csv('output.csv', index=False)

In [2]:
final.head(100)


,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"
...,...,...
95,95,"95, 0"
96,96,"96-9854-58319-66706-116460-356800, 11"
97,97,"97, 3"
98,98,"98, 0"
